### Buidling a question & answering application with GraphDB

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph(refresh_schema=False)

/home/crizz/python/rag_bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
### Dataset Movies
movie_query = """
    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' AS row
    
    MERGE (m:Movie{id:row.movieId})
    SET m.released = date(row.released),
      m.title = row.title,
      m.imdbRating = toFloat(row.imdbRating)
      
    FOREACH (director in split(row.director, '|') |
      MERGE (p:Person {name:trim(director)})
      MERGE (p)-[:DIRECTED]->(m))
      
    FOREACH (actor in split(row.actors, '|') | 
      MERGE (p:Person {name: trim(actor)})
      MERGE (p)-[:ACTED_IN]->(m))
      
    FOREACH (genre in split(row.genres, '|') |
      MERGE (g:Genre {name: trim(genre)})
      MERGE (m)-[:IN_GENRE]->(g))
"""

In [20]:
graph.query(movie_query)

[]

In [35]:
from langchain_ollama import ChatOllama

model = ChatOllama(model="google/gemma-3-4b-it:latest", reasoning=False)

In [36]:
from langchain_neo4j import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(
    graph=graph,
    llm=model,
    verbose=True,
    allow_dangerous_requests=True
)

In [37]:
chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x7240b676cd10>, cypher_generation_chain=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}')
| RunnableBinding(bound=ChatOllama(model='google/gemma-3-4b-it:latest', reasoning=False), kwargs={}, config={}, config_factories=[])
| StrOutputParser(), qa_chain=PromptTemplate(input_variables=['context', 'question'], input_types={}, partia

In [38]:
response = chain.invoke({'query':"Who was the director of the movie Casino use MATCH (p:Person)-[:DIRECTED]->(m:Movie {title: 'Casino'} RETURN p.name"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)-[:DIRECTED]->(m:Movie {title: 'Casino'}) RETURN p.name

Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': "Who was the director of the movie Casino use MATCH (p:Person)-[:DIRECTED]->(m:Movie {title: 'Casino'} RETURN p.name",
 'result': 'Martin Scorsese was the director of the movie Casino.'}